In [8]:
pip install ipywidget.chatbot

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:

pip install ipywidgets requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests, ipywidgets as widgets
from IPython.display import display

BASE_URL = "http://localhost:11434"   # Ollama native API
MODEL = "llama3.1:latest"             # your model

# UI widgets510 	1.143000
520 	1.080300
530 	1.062900
540 	1.119700
550 	1.029600
560 	1.053700
570 	1.049600
580 	1.041000
590 	1.183800
600 	1.049100
610 	1.084900
620 	1.062500
630 	1.082800
640 	1.125100
650 	1.016500
660 	1.079300
670 	1.060200
680 	1.022500
690 	1.148800
700 	1.152700
710 	1.059500
720 	1.082100
730 	1.010600
740 	0.993900
750 	1.095200
760 	1.037200
770 	1.044600
780 	1.097100
790 	1.067300
800 	0.997700
810 	1.105100
820 	0.985300
830 	1.085800
840 	1.110600
850 	0.983300
860 	1.020700
870 	0.949600
880 	1.131500
890 	0.965000
900 	1.048600
910 	1.078200
920 	1.035300
930 	1.097800
940 	1.136600
950 	1.064800
960 	0.994700
970 	0.965900
980 	1.069100
990 	1.062600
1000 	1.097000
1010 	1.082000
1020 	0.859700
1030 	0.928600
1040 	0.913200
1050 	0.862700
1060 	0.830500
1070 	0.884500
1080 	0.871900
1090 	0.936100
1100 	0.824400
1110 	0.928600
1120 	0.850000
1130 	0.887000
1140 	0.857600
1150 	0.837300
1160 	0.858600
1170 	0.871000
1180 	0.941700
1190 	0.882000
1200 	0.910300
1210 	0.786400
1220 	0.865000
1230 	0.924700
1240 	0.832300
1250 	0.880500
1260 	0.813600
1270 	0.847000
1280 	0.855400
1290 	0.895900
1300 	0.928200
1310 	0.839600
1320 	0.914500
1330 	0.796600
1340 	0.874000
1350 	0.887800
1360 	0.890800
1370 	0.907300
1380 	0.974700
1390 	0.866500
1400 	0.888200
1410 	0.916300
1420 	0.909300
1430 	0.908700
1440 	0.863200
1450 	0.886700
1460 	0.804600
1470 	0.868700
1480 	0.918700
1490 	0.919100
1500 	0.875700
1510 	0.867900
chat_log = widgets.Output(layout={'border': '1px solid gray',
                                  'height':'300px',
                                  'overflow_y':'auto'})
user_input = widgets.Textarea(
    placeholder="Type your message here...",
    layout={'width':'100%','height':'60px'}
)
send_button = widgets.Button(description="Send", button_style='primary')

def send_message(_):
    user_text = user_input.value.strip()
    if not user_text:
        return
    user_input.value = ""  # clear box

    # Show user text
    with chat_log:
        print(f"You: {user_text}")

    try:
        resp = requests.post(
            f"{BASE_URL}/api/chat",
            headers={"Content-Type": "application/json"},
            json={
                "model": MODEL,
                "messages": [{"role": "user", "content": user_text}],
                "stream": False
            },
            timeout=300
        )
        j = resp.json()
        # Ollama /api/chat returns {"message":{"role":"assistant","content":"..."}}
        reply = j.get("message", {}).get("content") or j.get("response") or "(no reply)"
    except Exception as e:
        reply = f"Error: {e}"

    # Show bot reply
    with chat_log:
        print(f"Bot: {reply}\n")

send_button.on_click(send_message)

display(chat_log, user_input, send_button)



Output(layout=Layout(border_bottom='1px solid gray', border_left='1px solid gray', border_right='1px solid gra…

Textarea(value='', layout=Layout(height='60px', width='100%'), placeholder='Type your message here...')

Button(button_style='primary', description='Send', style=ButtonStyle())

In [4]:
from transformers import GenerationConfig

# Start with a system prompt (optional)
history = [
    {"role": "system", "content": "You are a helpful assistant."}
]

def chat_step(user_text):
    global history
    # 1) add the user message
    history.append({"role": "user", "content": user_text})

    # 2) build input with chat template
    inputs = tok.apply_chat_template(
        history,
        add_generation_prompt=True,   # so model knows it's its turn
        return_tensors="pt"
    ).to(model.device)

    # 3) generate
    gen_cfg = GenerationConfig(
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )
    with torch.no_grad():
        out = model.generate(input_ids=inputs, generation_config=gen_cfg)

    # 4) decode only the new assistant part
    response = tok.decode(out[0][inputs.shape[-1]:], skip_special_tokens=True).strip()

    # 5) add assistant reply back into history
    history.append({"role": "assistant", "content": response})

    return response

# Example use
print("Bot:", chat_step("Hi, can you help me check my report?"))
print("Bot:", chat_step("Also, remind me what I asked before."))


NameError: name 'tok' is not defined

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# Path to your fine-tuned model folder
MODEL_PATH = "out-llama31-en-qlora"

# 1) Load tokenizer & model
tok = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
model.eval()

# 2) Keep conversation history
history = [
    {"role": "system", "content": "You are a helpful assistant."}
]

def chat_step(user_text):
    global history
    # Add user input
    history.append({"role": "user", "content": user_text})

    # Build full input with conversation history
    inputs = tok.apply_chat_template(
        history,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # Generate response
    gen_cfg = GenerationConfig(
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )
    with torch.no_grad():
        out = model.generate(input_ids=inputs, generation_config=gen_cfg)

    # Decode only the assistant's reply
    response = tok.decode(out[0][inputs.shape[-1]:], skip_special_tokens=True).strip()

    # Add assistant reply to history
    history.append({"role": "assistant", "content": response})

    return response

# Example usage
print("Bot:", chat_step("Hi, can you help me check my report?"))
print("Bot:", chat_step("Also, remind me what I asked before."))


OSError: out-llama31-en-qlora is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [6]:
import requests, ipywidgets as widgets
from IPython.display import display

BASE_URL = "http://localhost:11434"   # Ollama’s default
MODEL    = "llama3.1:latest"          # or "llama3.1:8b-instruct" if you have that tag

chat_log = widgets.Output(layout={'border':'1px solid gray','height':'300px','overflow_y':'auto'})
user_input = widgets.Textarea(placeholder="Type your message here...", layout={'width':'100%','height':'60px'})
send_button = widgets.Button(description="Send", button_style='primary')

def send_message(_):
    txt = user_input.value.strip()
    if not txt: return
    user_input.value = ""
    with chat_log: print(f"You: {txt}")

    try:
        r = requests.post(
            f"{BASE_URL}/api/chat",
            json={"model": MODEL, "messages":[{"role":"user","content":txt}], "stream": False},
            timeout=120
        )
        j = r.json()
        reply = j.get("message",{}).get("content") or j.get("response") or "(no reply)"
    except Exception as e:
        reply = f"Error: {e}"

    with chat_log: print(f"Bot: {reply}\n")

send_button.on_click(send_message)
display(chat_log, user_input, send_button)


Output(layout=Layout(border_bottom='1px solid gray', border_left='1px solid gray', border_right='1px solid gra…

Textarea(value='', layout=Layout(height='60px', width='100%'), placeholder='Type your message here...')

Button(button_style='primary', description='Send', style=ButtonStyle())

In [11]:
import json, os, requests, ipywidgets as widgets
from IPython.display import display

# --- Configure Ollama ---
BASE_URL = "http://localhost:11434"       # Ollama default
MODEL    = "llama3.1:latest"              # or your exact tag, e.g. "llama3.1:8b-instruct"

# --- Context controls ---
MAX_TURNS = 30        # keep the last N user+assistant exchanges (plus system)
SESSION_FILE = "ollama_session.json"  # persist across notebook restarts

# --- Load / init history ---
def load_history():
    if os.path.exists(SESSION_FILE):
        try:
            return json.load(open(SESSION_FILE, "r"))
        except Exception:
            pass
    # fresh start with a system message (tweak tone/persona as you like)
    return [{"role":"system","content":"You are a concise, helpful assistant."}]

def save_history(history):
    try:
        json.dump(history, open(SESSION_FILE, "w"))
    except Exception:
        pass

def trim_history(history, max_turns=MAX_TURNS):
    """
    Keep system + last N (user,assistant) turns.
    """
    sys = [m for m in history if m["role"]=="system"][:1]
    rest = [m for m in history if m["role"]!="system"]
    # each turn is 2 messages (user, assistant); keep last 2*MAX_TURNS
    keep = rest[-2*max_turns:]
    return sys + keep

history = load_history()

# --- UI ---
chat_log   = widgets.Output(layout={'border':'1px solid gray','height':'300px','overflow_y':'auto'})
user_input = widgets.Textarea(placeholder="Type your message here...", layout={'width':'100%','height':'70px'})
send_btn   = widgets.Button(description="Send", button_style='primary')
reset_btn  = widgets.Button(description="Reset memory", button_style='warning')

def send_message(_):
    global history
    user_text = user_input.value.strip()
    if not user_text:
        return
    user_input.value = ""

    # append user msg
    history.append({"role":"user","content":user_text})

    # trim before sending (protect context window)
    hist_to_send = trim_history(history)

    with chat_log:
        print(f"You: {user_text}")

    try:
        # Ollama chat API: https://github.com/ollama/ollama/blob/main/docs/api.md#chat
        r = requests.post(
            f"{BASE_URL}/api/chat",
            json={
                "model": MODEL,
                "messages": hist_to_send,
                "stream": False
            },
            timeout=300
        )
        r.raise_for_status()
        j = r.json()
        reply = j.get("message",{}).get("content") or j.get("response") or "(no reply)"
    except Exception as e:
        reply = f"Error: {e}"

    # append assistant msg & save/persist
    history.append({"role":"assistant","content":reply})
    history = trim_history(history)
    save_history(history)

    with chat_log:
        print(f"FouFlou:{reply}\n")

def reset_memory(_):
    global history
    history = [{"role":"system","content":"You are a concise, helpful assistant."}]
    save_history(history)
    with chat_log:
        print("— memory cleared —\n")

send_btn.on_click(send_message)
reset_btn.on_click(reset_memory)
display(chat_log, user_input, widgets.HBox([send_btn, reset_btn]))


Output(layout=Layout(border_bottom='1px solid gray', border_left='1px solid gray', border_right='1px solid gra…

Textarea(value='', layout=Layout(height='70px', width='100%'), placeholder='Type your message here...')